In [2]:
import os

from args import *
from train_test_functions import *
from model import *
from data import *

In [3]:
args = Args(cuda=torch.cuda.is_available(), graph_name='ENZYMES')

In [4]:
args.epochs = 2000
args.batch_size = 128
args.reco_importance = 0.1
args.loss = nn.BCELoss()

os.environ['CUDA_VISIBLE_DEVICES'] = str(args.cuda)
print('CUDA', args.cuda)

CUDA True


In [7]:
graphs = graph_load_batch(data_directory=args.data_directory, name=args.graph_name)

dataloaders_train, dataloaders_test = create_loaders(graphs, args)

Loading graph dataset: ENZYMES
Loaded
total graph num: 600, training set: 540
max number node: 125
max/min number edge: 149; 1
max previous node: 25


In [8]:
results = {}

args.num_fold = None

for i in range(10):

    print('Fold number: {:.0f}'.format(i+1))
    args.num_fold = i

    rnn_embedding = RecurrentEmbedding(input_size=args.node_dim, 
                                       embedding_size=args.embedding_size_rnn,
                                       hidden_size=args.hidden_size_rnn, 
                                       num_layers=args.num_layers, 
                                       is_cuda=args.cuda)

    var = VAR(h_size=args.hidden_size_rnn, 
              embedding_size=args.embedding_size_output,
              y_size=args.node_dim, 
              is_cuda=args.cuda)

    rnn_classifier = RecurrentClassifier(input_size=args.hidden_size_rnn, 
                                         embedding_size=args.embedding_size_rnn,
                                         hidden_size=args.hidden_size_rnn, 
                                         num_layers=args.num_layers, 
                                         num_class=args.num_class,
                                         is_cuda=args.cuda)

    if args.cuda:
        rnn_embedding = rnn_embedding.cuda()
        var = var.cuda()
        rnn_classifier = rnn_classifier.cuda()

    learning_accuracy_test = classifier_train(args, 
                                              dataloaders_train[i], 
                                              dataloaders_test[i], 
                                              rnn_embedding, var, rnn_classifier)

    accuracy_test, scores, predicted_labels, true_labels, vote = vote_test(args, 
                                                                           rnn_embedding, 
                                                                           var, 
                                                                           rnn_classifier,
                                                                           dataloaders_test[i], 
                                                                           num_iteration=100)
    
    results[i] = {'rnn': rnn_embedding, 'output': var, 'classifier_1': rnn_classifier,
                  'acc_test': accuracy_test, 'scores': scores}

print([results[r]['acc_test'] for r in results])

Fold number: 0
Dataset: ENZYMES, Epoch: 1/10, train bce loss: 0.364, train kl loss: 0.012, classifier loss: 9.021
Dataset: ENZYMES, Epoch: 2/10, train bce loss: 0.231, train kl loss: 0.019, classifier loss: 8.995
Dataset: ENZYMES, Epoch: 3/10, train bce loss: 0.162, train kl loss: 0.031, classifier loss: 8.965
Dataset: ENZYMES, Epoch: 4/10, train bce loss: 0.135, train kl loss: 0.039, classifier loss: 8.974
Dataset: ENZYMES, Epoch: 5/10, train bce loss: 0.133, train kl loss: 0.042, classifier loss: 8.955
Dataset: ENZYMES, Epoch: 6/10, train bce loss: 0.130, train kl loss: 0.039, classifier loss: 8.939
Dataset: ENZYMES, Epoch: 7/10, train bce loss: 0.124, train kl loss: 0.035, classifier loss: 8.911
Dataset: ENZYMES, Epoch: 8/10, train bce loss: 0.121, train kl loss: 0.032, classifier loss: 8.906
Dataset: ENZYMES, Epoch: 9/10, train bce loss: 0.122, train kl loss: 0.030, classifier loss: 8.863
Dataset: ENZYMES, Epoch: 10/10, train bce loss: 0.119, train kl loss: 0.029, classifier loss: 

Dataset: ENZYMES, Epoch: 3/10, train bce loss: 0.167, train kl loss: 0.030, classifier loss: 8.956
Dataset: ENZYMES, Epoch: 4/10, train bce loss: 0.141, train kl loss: 0.036, classifier loss: 8.933
Dataset: ENZYMES, Epoch: 5/10, train bce loss: 0.126, train kl loss: 0.040, classifier loss: 8.909
Dataset: ENZYMES, Epoch: 6/10, train bce loss: 0.125, train kl loss: 0.040, classifier loss: 8.900
Dataset: ENZYMES, Epoch: 7/10, train bce loss: 0.121, train kl loss: 0.038, classifier loss: 8.878
Dataset: ENZYMES, Epoch: 8/10, train bce loss: 0.114, train kl loss: 0.036, classifier loss: 8.858
Dataset: ENZYMES, Epoch: 9/10, train bce loss: 0.119, train kl loss: 0.032, classifier loss: 8.830
Dataset: ENZYMES, Epoch: 10/10, train bce loss: 0.117, train kl loss: 0.029, classifier loss: 8.788
Fold number: 9
Dataset: ENZYMES, Epoch: 1/10, train bce loss: 0.302, train kl loss: 0.024, classifier loss: 8.982
Dataset: ENZYMES, Epoch: 2/10, train bce loss: 0.190, train kl loss: 0.035, classifier loss: 

In [9]:
print(np.mean([results[r]['acc_test'] for r in results]), 
      np.std([results[r]['acc_test'] for r in results]))

0.2583333333333333 0.03095695936834451
